### Structure Learning

In [1]:
import pandas as pd
import numpy as np

Converts categorical data to numerical indices for machine learning compatibility.

1. **Data Preservation**: Creates a copy of the input DataFrame to avoid altering the original.
2. **Universal Encoding**: Processes all columns, converting categorical values (e.g., `["A", "B"]`) to integer codes (`[0, 1]`).
3. **Factorization**: Uses `pandas.factorize()[0]` to map unique categories to sequential integers.
4. **Return**: Outputs a fully encoded DataFrame (e.g., `["Red", "Blue"]` → `[0, 1]`). *Does not store encoder mappings.*

In [2]:
def encode_data(df):
    encoded = df.copy()
    for col in df.columns:
        encoded[col] = encoded[col].factorize()[0]
    return encoded

Prepares mushroom dataset for machine learning.

1. **Data Loading**: Reads training/test CSVs (`mushroom_train.data`, `mushroom_test.data`) without headers.
2. **Categorical Encoding**: Applies `encode_data` to convert all columns to numerical indices.
3. **Feature-Target Split**:
   - Features (`X`): All columns except the first (index `1:`).
   - Target (`y`): First column (index `0`).
4. **Array Conversion**: Converts DataFrames to NumPy arrays (`.values`) for model compatibility.
5. **Feature Count**: Extracts number of features (`n_features = X_train.shape[1]`).

In [3]:
# Load and preprocess data
train_df = pd.read_csv('mushroom_train.data', header=None)
test_df = pd.read_csv('mushroom_test.data', header=None)

train_encoded = encode_data(train_df)
test_encoded = encode_data(test_df)

X_train = train_encoded.iloc[:, 1:].values
y_train = train_encoded.iloc[:, 0].values
X_test = test_encoded.iloc[:, 1:].values
y_test = test_encoded.iloc[:, 0].values

n_features = X_train.shape[1]

Quantifies feature dependencies conditioned on class labels for structure learning.

1. **Matrix Initialization**: Creates empty `n_features x n_features` matrix (`cmi_matrix`).
2. **Feature Pair Iteration**: Processes all unique feature pairs `(i, j)`.
3. **Class-Conditional Calculation**: For each class `y` in target:
   - **Data Filtering**: Extracts feature values `x_i` and `x_j` for class `y`.
   - **Joint Probability**: Computes `joint` distribution via contingency table (`pd.crosstab`).
   - **Marginal Probabilities**: Calculates `p_i` and `p_j` using value counts.
   - **Mutual Information**: Accumulates `mi` using `Σ joint * log(joint/(p_i*p_j))` (with `1e-10` smoothing).
4. **CMI Aggregation**: Weights `mi` by class frequency and sums across classes.
5. **Symmetry**: Assigns `cmi` to both `[i,j]` and `[j,i]` for symmetric matrix.

In [4]:
cmi_matrix = np.zeros((n_features, n_features))
for i in range(n_features):
    for j in range(i + 1, n_features):
        cmi = 0
        for y in np.unique(y_train):
            mask = y_train == y
            x_i = X_train[mask, i]
            x_j = X_train[mask, j]

            joint = pd.crosstab(x_i, x_j).values / len(x_i)

            p_i = np.bincount(x_i) / len(x_i)
            p_j = np.bincount(x_j) / len(x_j)

            mi = 0
            for xi in range(joint.shape[0]):
                for xj in range(joint.shape[1]):
                    if joint[xi, xj] > 0:
                        mi += joint[xi, xj] * np.log(joint[xi, xj] / (p_i[xi] * p_j[xj] + 1e-10))
            cmi += (np.sum(mask) / len(y_train)) * mi
        cmi_matrix[i, j] = cmi_matrix[j, i] = cmi

Constructs a tree structure capturing the strongest feature dependencies for Bayesian network learning. Uses negative CMI input to effectively compute a Maximum Spanning Tree.
1. **Edge List Creation**: Extracts non-zero edges from `neg_cmi`, stores them as `(-value, i, j)` to invert weights.
2. **Descending Sort**: Orders edges by weight (high to low) to prioritize strong dependencies.
3. **Union-Find Setup**: Initializes parent array for cycle detection during tree construction.
4. **Kruskal's Algorithm**:
   - Iterates sorted edges, uses `find` with path compression to check connectivity.
   - Adds edges to MST if they connect disjoint components.
5. **Weight Restoration**: Stores original CMI values (via `-weight`) in the MST matrix.

In [5]:
def minimum_spanning_tree(neg_cmi):
    n = neg_cmi.shape[0]
    edges = []
    for i in range(n):
        for j in range(i + 1, n):
            if neg_cmi[i, j] != 0:
                edges.append((-neg_cmi[i, j], i, j))

    edges.sort(reverse=True)
    parent = list(range(n))

    def find(u):
        while parent[u] != u:
            parent[u] = parent[parent[u]]
            u = parent[u]
        return u

    mst = np.zeros((n, n))
    for weight, u, v in edges:
        root_u = find(u)
        root_v = find(v)
        if root_u != root_v:
            mst[u, v] = -weight
            parent[root_v] = root_u
    return mst

Transforms the CMI matrix into a Maximum Spanning Tree (MST) by negating values (to use Kruskal's algorithm for MST), prioritizing edges with highest conditional mutual information. Returns a tree structure of strongest feature dependencies for Bayesian network learning.

In [6]:
mst = minimum_spanning_tree(-cmi_matrix)

Creates a directed tree structure from the MST, rooted at the feature most predictive of the target.
1. **Root Selection**:
   - Computes mutual information (`mi_with_y`) between each feature and target `y` using joint/marginal probabilities.
   - Selects root as the feature with maximum MI (`root = np.argmax(mi_with_y)`).

2. **Tree Initialization**:
   - `parent` array initialized to `-1` (no parent).
   - BFS setup: `visited` set and `queue` start with root node.

3. **BFS Tree Building**:
   - Iterates through MST adjacency matrix (`mst`).
   - Assigns parent-child relationships for unvisited neighbors.
   - Expands tree until all nodes are processed.

In [7]:
mi_with_y = []
for i in range(n_features):
    contingency = pd.crosstab(y_train, X_train[:, i]).values
    p_joint = contingency / contingency.sum()
    p_y = p_joint.sum(axis=1)
    p_x = p_joint.sum(axis=0)
    mi = np.sum(p_joint * np.log((p_joint + 1e-10) / (np.outer(p_y, p_x) + 1e-10)))
    mi_with_y.append(mi)
root = np.argmax(mi_with_y)

parent = np.full(n_features, -1)
visited = set([root])
queue = [root]

while queue:
    current = queue.pop(0)
    neighbors = np.where(mst[current] > 0)[0]
    for neighbor in neighbors:
        if neighbor not in visited:
            parent[neighbor] = current
            visited.add(neighbor)
            queue.append(neighbor)

Nested CPTs storing probability distributions for Bayesian network inference.
1. **Node Initialization**: Creates empty CPTs for each feature.
2. **Root Handling**: For root nodes (no parent):
   - Computes `P(feature|class)` using class-filtered counts + Laplace smoothing.
3. **Child Nodes**: For nodes with parents:
   - Computes `P(feature|class, parent_value)` using parent-value-filtered counts + smoothing.
4. **Probability Calculation**:
   - **Numerator**: Observed counts + 1 (smoothing).
   - **Denominator**: Total samples + num_categories (normalization).

In [8]:
cpt = {}
for feat in range(n_features):
    cpt[feat] = {}
    parent_feat = parent[feat]

    for y in np.unique(y_train):
        mask = y_train == y
        if parent_feat == -1:  # Root node depends only on Y
            counts = np.bincount(X_train[mask, feat], minlength=len(np.unique(X_train[:, feat])))
            cpt[feat][y] = (counts + 1) / (counts.sum() + len(np.unique(X_train[:, feat])))
        else:
            parent_vals = X_train[mask, parent_feat]
            unique_parents = np.unique(parent_vals)
            cpt[feat][y] = {}
            for p_val in unique_parents:
                sub_mask = parent_vals == p_val
                counts = np.bincount(X_train[mask, feat][sub_mask],
                                     minlength=len(np.unique(X_train[:, feat])))
                cpt[feat][y][p_val] = (counts + 1) / (counts.sum() + len(np.unique(X_train[:, feat])))

**Prediction Workflow**:
1. **Class Probability Initialization**:
   - Computes log prior probability with Laplace smoothing: `log((count(y) + 1) / (N + num_classes))`.

2. **Feature Probability Accumulation**:
   - **Root Features**: Directly accesses `P(feature|class)` from CPT.
   - **Child Features**: Uses `P(feature|class, parent_value)` from CPT.
   - **Unseen Parent Values**: Defaults to uniform distribution over feature categories.

3. **Log-Probability Stability**:
   - Adds `1e-10` offset before taking `log()` to avoid numerical errors.

4. **Final Prediction**: Returns class with maximum accumulated log-probability.

**Key Flow**: `log(prior) + Σ log(P(feature|class, parents)) → argmax`

In [9]:
def predict(X):
    log_probs = []
    for y in np.unique(y_train):
        log_prob = np.log((np.sum(y_train == y) + 1) / (len(y_train) + len(np.unique(y_train))))

        for feat in range(n_features):
            if parent[feat] == -1:
                prob = cpt[feat][y][X[feat]]
            else:
                p_val = X[parent[feat]]
                num_categories = len(np.unique(X_train[:, feat]))
                default_prob = np.ones(num_categories) / num_categories
                prob = cpt[feat][y].get(p_val, default_prob)[X[feat]]

            log_prob += np.log(prob + 1e-10)

        log_probs.append(log_prob)
    return np.unique(y_train)[np.argmax(log_probs)]

Quantifies model performance on unseen data using the Chow-Liu Bayesian network structure.
1. **Prediction Generation**: Applies the `predict` function to every test sample (`X_test`), generating predicted labels (`y_pred`).
2. **Accuracy Calculation**: Compares predicted labels with ground truth (`y_test`), computes correct prediction ratio using `np.mean()`.
3. **Result Output**: Prints accuracy score formatted to 4 decimal places (e.g., `Chow-Liu Accuracy: 0.9532`).

In [10]:
y_pred = np.array([predict(x) for x in X_test])
accuracy = np.mean(y_pred == y_test)
print(f"Chow-Liu Accuracy: {accuracy:.4f}")

Chow-Liu Accuracy: 0.8949
